## ESCAPE 4 JUNE 2022

### Lightning for all RHIs in southern cell scanned by PX1000

In [1]:
import matplotlib.pyplot as plt
import pyart
import numpy as np
import cartopy.crs as ccrs
import pandas as pd
import xarray as xr
from datetime import datetime, date, time, timedelta
import pyart.graph.cm as pcm
import warnings
warnings.filterwarnings("ignore")

from pyxlma.coords import RadarCoordinateSystem, GeographicSystem, TangentPlaneCartesianSystem, MapProjection



px_elevation_m = 318


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



### [LMAinterceptRHI](https://github.com/jcssouza/LMAinterceptRHI)

In [2]:
import sys
sys.path.append("./LMAinterceptRHI/")
# Adapted functions from LMAinterceptRHI
from radarlma2local import geo_to_tps
from ortho_proj import rot_mat_lma, close_sources, closest_pt_radarloc
from radar_processing import r_z_centers_edges_mesh
from interp_funcs import centers_to_edges_1d, coords_2d

In [3]:
from shapely.geometry import Polygon, Point, MultiPoint

class RadarLMASubsetter:
    def __init__(self, radar, lma, sweep_id=0):
        self.radar = radar
        self.lma = lma
        
        # -- Get radar time
        self.radar_time = datetime.utcfromtimestamp(radar.Time)
        
        print(self.radar_time)
        
        # -- Initialize coordinate systems
        ctrlat, ctrlon, ctralt = radar.Latitude, radar.Longitude, px_elevation_m
        self.ctrlat, self.ctrlon, self.ctralt = ctrlat, ctrlon, ctralt
        self.geo = GeographicSystem()
        self.tps = TangentPlaneCartesianSystem(ctrlat, ctrlon, ctralt)
        self.rcs = RadarCoordinateSystem(ctrlat, ctrlon, ctralt)



        # -- Radar scan geometry: centers and edges of each gate
        RZ_centers, RZ_edges = r_z_centers_edges_mesh(radar, px_elevation_m)
        self.r_c = RZ_centers[:,:,0]
        self.z_c = RZ_centers[:,:,1]
        self.r_e = RZ_edges[:,:,0]
        self.z_e = RZ_edges[:,:,1]

        # Radar shape
        # -- To filter VHF sources out of the radar scan domain
        # --- Create the polygon: all ranges, lowest and highest elevation
        self.scan_poly = Polygon(np.append(
                np.append(RZ_edges[-1, :-1], RZ_edges[0, :-1], axis = 0), 
                RZ_edges[:, - 1], axis = 0))
        # scan_poly

    
        # -- Radar scan geometry: centers and edges of each gate
        rng = np.matmul(radar.Gate.data.reshape(-1, 1), radar.GateWidth.data.reshape(1, -1))
        az = np.tile(radar.Azimuth.data, (rng.shape[0], 1))
        eldat = np.tile(radar.Elevation.data, (rng.shape[0], 1))

        ecef_X, ecef_Y, ecef_Z = self.rcs.toECEF(rng, az, eldat)
        tpcs_x, tpcs_y, tpcs_z = self.tps.fromECEF(ecef_X, ecef_Y, ecef_Z)

        tpcs_x = tpcs_x.reshape(rng.shape)
        tpcs_y = tpcs_y.reshape(rng.shape)
        self.z_c = tpcs_z.reshape(rng.shape)

        self.r_c = (tpcs_x**2 + tpcs_y**2)**0.5    

        self.gate_lon, self.gate_lat, self.gate_alt = self.geo.fromECEF(ecef_X, ecef_Y, ecef_Z)


        # Radar shape
        # -- To filter VHF sources out of the radar scan domain
        # --- Create the polygon: all ranges, lowest and highest elevation
        self.scan_poly = Polygon(np.append(
                np.append(RZ_edges[-1, :-1], RZ_edges[0, :-1], axis = 0), 
                RZ_edges[:, - 1], axis = 0))
        # scan_poly
        
    
    def lightning_within_time(self, seconds, max_alt=18e3, max_chi2=1.0, min_station=6):
        lma_data = self.lma
        radar_time = self.radar_time
        
        # Filter time and altitude
        sources = (lma_data.event_altitude < max_alt) & (abs( (np.datetime64(radar_time) - lma_data.event_time)/ np.timedelta64(1, 's') ) < seconds) 
        subset = lma_data[{'number_of_events': sources}]

        flashes_ids = np.unique(subset.event_parent_flash_id).astype(int)
        print(f'{len(flashes_ids)} flashes in {2*seconds} second interval near scan.')

        
        # Select events info for individual flash id ('number_of_events' dimension)
        # nid = 64
        # one_id = lma_data.event_parent_flash_id == flashes_ids[nid]
        # flash_events = lma_data[{'number_of_events': one_id}]

        # Filter data
        filter_events = (subset.event_chi2 < max_chi2) & (subset.event_stations > min_station)
        filter_events
        filtered = subset[{'number_of_events':filter_events}]

        # Only one flash events info in the dataset
        all_dims = dict(filtered.dims)
        all_dims.pop('number_of_events') #all dims will no longer have 'number of flashes’
        return filtered.drop_dims(all_dims.keys())

    def plot_lightning_near_scan_time(self, seconds,  **kwargs):
        radar = self.radar
        
        events_near = self.lightning_within_time(seconds, **kwargs)
        # Quick look at RHI relative to the selected lightning before coordinate transformations
        fig, ax = plt.subplots(1,1)
        flash_time = np.sort(events_near.event_time)
        loc_lma = ax.scatter(events_near.event_longitude, events_near.event_latitude,
                              marker = '.', s = 9, c = (flash_time - flash_time[0]), cmap = 'viridis',alpha=0.9)
        loc_radar = ax.scatter(radar.Longitude,radar.Latitude,
                                color = 'brown', marker = 's', s = 60)
        loc_rhi = ax.scatter(self.gate_lon, self.gate_lat, s = 3, color = 'black')
        return ax
        # plt.axis((-95.5, -95.3, 29.65, 29.85))

    def find_lightning_distances_near_scan_time(self, seconds, distance, **kwargs):
        events_near = self.lightning_within_time(seconds, **kwargs)
        radar = self.radar

        # --- Orthogonal projection/ Matrix Rotation of the LF sources
        Xlma,Ylma,Zlma = geo_to_tps(events_near, radar, px_elevation_m)   # Filter does not have altitude
        XYZlma = np.column_stack((Xlma, Ylma, Zlma))
        lma_file_ortho = rot_mat_lma(radar, XYZlma, -1)  # -1 for counterclockwise

        # 1st STEP
        # -- Find and store sources within a certain ds distance
        ds = distance # m threshold
        self.ds = ds
        r_cls, z_cls, y_min = close_sources(self.r_c, self.z_c, lma_file_ortho, ds)
        print(np.min(y_min))


        close_mask = np.abs(y_min) < ds
        return r_cls[close_mask], z_cls[close_mask], y_min[close_mask]
        # -- Selected LMA sources inside the radar scan
        lma_shp = MultiPoint(tuple(np.vstack((r_cls, z_cls)).transpose()))
        R_cls = [] 
        Z_cls = []
        Y_min = []
        for i in np.arange(len(r_cls)):
            if self.scan_poly.contains(lma_shp) == True:
                R_cls.append(np.asarray(lma_shp.coords[0])[0])
                Z_cls.append(np.asarray(lma_shp.coords[0])[1])
                Y_min.append(y_min[i])
        R_cls = np.asarray(R_cls)
        Z_cls = np.asarray(Z_cls)
        Y_min = np.asarray(Y_min)
        
        return R_cls, Z_cls, Y_min
    
    def radar_gate_at_lightning_source(self, R_cls, Zcls, Y_min):
        
        radar = self.radar
        
        # ASSUME Y = 0 is the location of the scan plane
        int_point1 = [np.array([R_cls[i], 0 , Z_cls[i]]) for i in np.arange(R_cls.size)]

        # 2o STEP
        # Transform from Rotationed Coordinate system Tangent Plane to Tangent Plan (rotates clockwise = 1)
        int_point2 = rot_mat_lma(radar, int_point1, 1)

        # 3o STEP
        # Tangent Plan to ECEF
        int_point3 = [self.tps.fromLocal(int_point2[i,:][:,None]) for i in np.arange(R_cls.size)]

        # 4o STEP
        # Transform from ECEF to Radar Coordinate System
        int_point4 = [self.rcs.fromECEF(int_point3[i][0],int_point3[i][1],int_point3[i][2]) for i in np.arange(R_cls.size)]

        # 5o STEP
        # Find closest point to RHI - Elevation and Range
        cls_r_idx = [closest_pt_radarloc(radar, int_point4[i])[0]  for i in np.arange(R_cls.size)]    # Radius index
        cls_az_idx = [closest_pt_radarloc(radar, int_point4[i])[1]  for i in np.arange(R_cls.size)]   # Azimuth index
        cls_elev_idx = [closest_pt_radarloc(radar, int_point4[i])[2]  for i in np.arange(R_cls.size)] # Elevation index
        
        return cls_r_idx, cls_az_idx, cls_elev_idx

In [4]:
class OverlayPlotter:
    
    def __init__(self, subsetter, seconds, distance):
        self.subsetter = subsetter
        self.radar = subsetter.radar
        self.ranges, self.z = subsetter.r_c, subsetter.z_c
        self.R_cls, self.Z_cls, self.Y_min = subsetter.find_lightning_distances_near_scan_time(seconds, distance)
        self.ds = subsetter.ds
        
    def plot(self, radar, field, distance_max = 1000):
        
        cbar_range = {
            'Intensity' : (-10, 80),
            'Radial_Velocity' : (-radar.attrs['Nyquist_Vel-value'], -radar.attrs['Nyquist_Vel-value']),
            'Differential_Reflectivity' : (-2, 8),
            'RhoHV' : (0, 1),
            'Width' : (0, 12.5),
            'PhiDP' : (-180, 180)
        }
        cmaps = {
            'Intensity' : 'pyart_ChaseSpectral',
            'Radial_Velocity' : 'pyart_balance',
            'Differential_Reflectivity' : 'pyart_turbone_zdr',
            'RhoHV' : 'pyart_plasmidis',
            'Width' : 'cubehelix_r',
            'PhiDP' : 'pyart_Wild25'
        }
        units={
                'Intensity':'Reflectivity (dBZ)',
                'Radial_Velocity':'Velocity (m/s)',
                'Differential_Reflectivity':'Differential Reflecivity (dB)',
                'RhoHV':'Correlation Coeff. (unitless)',
                'Width':'Spectrum width (m/s)',
                'ZDR':'Differential Reflectivity (dB)',
                'PhiDP':'Differential Phase (deg)'
                }

        R_cls, Z_cls, Y_min = self.R_cls, self.Z_cls, self.Y_min
        
        # -- For plots below
        # -- Selecting coordinates and values for images below
        values = np.ma.masked_array(radar[field].data, radar[field].data <= -99900.0).T

        if field in cbar_range:
            vmin, vmax = cbar_range[field]
        else:
            vmin, vmax = np.nanmin(values), np.nanmax(values)
        fig = plt.figure(figsize=(11, 7.5), dpi=180)
        # ax1 = fig.add_subplot(111)
        
        left, bottom = 0.05, 0.08
        margin = left/1.8
        height = 0.8
        mainwidth = 0.75
        cbarwidth = 0.02
        
        ax1 = fig.add_axes([left, bottom, mainwidth, bottom+height])
        cs = ax1.pcolormesh(self.ranges/1000, self.z/1000, values, vmin = vmin, vmax = vmax, cmap = cmaps[field])
        llma = plt.scatter(R_cls/1000,  Z_cls/1000, c = abs(Y_min), 
                           edgecolor = 'black', cmap = 'Reds_r', marker = "X", 
                           s = 200, alpha = 0.5, vmin = 0, vmax = distance_max, zorder=10)
        print(R_cls.shape)
        # -- cbar
        cb_ax = fig.add_axes([left+mainwidth+margin/2, bottom, cbarwidth, bottom+height])
        cbar = fig.colorbar(cs, cax=cb_ax)
        cbar.set_label(units[field])#, size=15)
        # cbar.ax.tick_params(labelsize=11)

        llma_ax = fig.add_axes([left+mainwidth+margin/2+cbarwidth+margin*3, bottom, cbarwidth, bottom+height])
        cbar2 = fig.colorbar(llma, cax=llma_ax)
        cbar2.set_label('Orthogonal Distance (m)') #, size=15)
        # cbar2.ax.tick_params(labelsize=11)

        # fig.text(0.55, 0.9,self.subsetter.radar_time.strftime("%d %B %Y %H:%M:%S") + 
        #          f' - VHF sources within {ds} m from the RHI scan', 
        #          va='center', ha='center', fontsize = 15)
        # fig.text(0.5, 0.05, 'Distance from radar (km)', va='center', ha='center', fontsize = 15)
        # fig.text(0.09, 0.5, 'Distance above radar (km)', va='center', ha='center', rotation='vertical', fontsize = 15)
        ax1.set_ylim(0, 13)
        ax1.set_title(self.subsetter.radar_time.strftime("%d %B %Y %H:%M:%S") + 
                 f' - VHF sources within {self.ds} m from the RHI scan', )
                 # va='center', ha='center', fontsize = 15)
        ax1.set_xlabel('Distance from radar (km)')#, va='center', ha='center', fontsize = 15)
        ax1.set_ylabel('Distance above radar (km)')#, va='center', ha='center', fontsize = 15)

        return fig

### Lightning

One file has the whole day.

In [5]:
# Read lma file
lightning_file = '../../lma/0616/LYLOUT_230616_013000_0600.nc'
lma_data = xr.open_dataset(lightning_file)
# lma_data.event_time.values = pd.to_datetime(lma_data.event_time.values.astype('M8[us]')).to_pydatetime()

### Subsetting and plotting

In [6]:
import os
from pathlib import Path
figures_out = './cases/figures/0615rhi'

Path(figures_out).mkdir(parents=True, exist_ok=True)

from glob import glob


In [7]:




units={'reflectivity':'Reflectivity (dBZ)',
    'differential_reflectivity':'Differential Reflecivity (dB)',
    'copol_correlation_coeff':'Correlation Coeff. (unitless)',
    'normalized_coherent_power':'Normalized Coherent Power (dB)',
    'specific_differential_phase':'Specific Differential Phase (deg/km)',
    'KDP_CSU':'Specific Differential Phase (deg/km)',
    'DBZ':'Reflectivity (dBZ)', 
    'VEL':'Velocity (m/s)',
    'WIDTH':'Spectrum width (m/s)',
    'ZDR':'Differential Reflectivity (dB)',
    'RHOHV':'Correlation Coefficient (unitless)',
    'PHIDP':'Differential Phase (deg)',
   }

# Defaults to min, max
# cbar_range={'reflectivity':(0,65),
#     'differential_reflectivity':(-1, 5),
#     'copol_correlation_coeff':(.90, 1.0),
#     'KDP_CSU':(-15, 15),
#    }
cbar_range={'DBZ':(-10,80),
    'ZDR':(-1, 5),
    'VEL':(-30, 30),
    'WIDTH':(-10, 10),
    'PHIDP':(-90, 90),
    'RHOHV':(.80, 1.0),
   }

seconds = 45
ds = 1000

In [9]:
def plot_one_radar_file(figure_path, radar_filename):
    radar = xr.open_dataset(radar_filename)
    # print(radar_filename,radar.scan_type)
    # print(list(radar.fields.keys()))
    subsetter = RadarLMASubsetter(radar,lma_data)
    plotter = OverlayPlotter(subsetter, seconds, ds)
    
    
    outfile_time_base = os.path.join(figure_path, 
                                        subsetter.radar_time.strftime('%Y%m%d_%H%M%S.'))
                            
    # fig = plotter.plot('normalized_coherent_power', units=units,
    #     cbar_range=cbar_range, distance_max=ds)
    # outfilename = outfile_time_base + 'normalized_coherent_power' + '.png'
    # fig.savefig(outfilename)
    # plt.close(fig)
    
    # fields_to_plot = ['reflectivity', 'differential_reflectivity',
    #              'specific_differential_phase', 'KDP_CSU',
    #              'copol_correlation_coeff']
    
    azimuth = np.median(radar.Azimuth.data)
                        
    # plan_ax = subsetter.plot_lightning_near_scan_time(seconds)
    # plan_ax.axis((subsetter.ctrlon-0.3, subsetter.ctrlon+0.3, subsetter.ctrlat-0.3, subsetter.ctrlat+0.3))
    # plan_ax.figure.savefig(outfile_time_base + f'az{azimuth:03.0f}.' + 'plan.png')
    # plt.close(plan_ax.figure)

    for datatype in ['-Z', '-D', '-R']: 
        this_field_fname = radar_filename.replace('-Z.nc', datatype + '.nc')
        radar = xr.open_dataset(this_field_fname)
        field = radar.TypeName        
        outfilename = outfile_time_base + f'az{azimuth:03.0f}.' + field + '.png'

        fig = plotter.plot(radar, field, distance_max=ds)
        fig.savefig(outfilename)
        plt.close(fig)
    return
                  
all_radar_files = sorted(glob("../../px1k/rhi/PX-*-A*-Z.nc"))
for radar_file in all_radar_files:
    plot_one_radar_file(figures_out, radar_file)

2023-06-16 01:34:50
7585 flashes in 90 second interval near scan.
x-dist = -183729.88405775971
y-dist = -7.669007532691467
x-dist = -167124.9726151509
y-dist = 914.949448874846
x-dist = -182386.27058618102
y-dist = -362.9437922942161
x-dist = -158101.3828643699
y-dist = 612.2563219373405
x-dist = -169123.10350287397
y-dist = 582.2942633749917
-997.745509740349
(1384,)
(1384,)
(1384,)
2023-06-16 01:35:02
7677 flashes in 90 second interval near scan.
x-dist = -183729.88405775971
y-dist = -7.669007532691467
x-dist = -167124.9726151509
y-dist = 914.949448874846
x-dist = -182386.27058618102
y-dist = -362.9437922942161
x-dist = -158101.3828643699
y-dist = 612.2563219373405
x-dist = -169123.10350287397
y-dist = 582.2942633749917
x-dist = -166979.7790746835
y-dist = -470.08304653050436
-997.745509740349
(1256,)
(1256,)
(1256,)


In [9]:
radar = xr.open_dataset('../../px1k/PX-20230616-000001-A95.0-D.nc')

In [10]:
# Does not provide a way to get the indices of the points inside the scan,
# so you lose the link to the third dimension.
# lma_in_rhi = lma_shp.intersection(scan_poly)
# for geom in lma_in_rhi.geoms:
#     print(geom)

In [11]:
# Example of finding nearby sources and their closest radar gate.
# R_cls, Z_cls, Y_min = subsetter.find_lightning_distances_near_scan_time(30, 1000)
# cls_r_idx, cls_az_idx, cls_elev_idx = subsetter.radar_gate_at_lightning_source(R_cls, Z_cls, Y_min)